In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
#import scipy.stats as stats
#import statsmodels.api as sm
import re
import plotly.express as px
import requests
from bs4 import BeautifulSoup as BS
from io import StringIO
from shapely.geometry import Point
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
%matplotlib inline

In [2]:
wiki = pd.read_csv('../data/wiki.csv')

In [3]:
wiki

,Unnamed: 0,origin,country,lat,long,depth_km,magnitude,secondary_effects,pde_shaking_deaths,pde_total_deaths,utsu_total_deaths,emdat_total_deaths,deaths,magnitude.1,measure
0,0,1900-05-11 17:23,Japan,38.700,141.100,5,7.0 MJMA,NaN,NaN,NaN,NaN,NaN,NaN,7.0,MJMA
1,1,1900-07-12 06:25,Turkey,40.300,43.100,NaN,5.9 Muk,NaN,NaN,NaN,140.0,NaN,140.0,5.9,Muk
2,2,1900-10-29 09:11,Venezuela,11.000,-66.000,0,7.7 Mw,NaN,NaN,NaN,NaN,NaN,NaN,7.7,Mw
3,3,1901-02-15 00:00,China,26.000,100.100,0,6.5 Ms,NaN,NaN,NaN,NaN,NaN,NaN,6.5,Ms
4,4,1901-03-31 07:11,Bulgaria,43.400,28.700,NaN,6.4 Muk,NaN,NaN,NaN,4.0,NaN,4.0,6.4,Muk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335,1335,2011-03-24 20:25,Burma (see 2011 Burma earthquake),NaN,NaN,NaN,6.8 Mw,NaN,NaN,NaN,NaN,NaN,NaN,6.8,Mw
1336,1336,2011-04-07 14:32,Japan (see April 2011 Miyagi earthquake),38.2,140.0,66.0,7.1 Mw,NaN,NaN,NaN,NaN,NaN,NaN,7.1,Mw
1337,1337,2011-09-18 12:40,India (see 2011 Sikkim earthquake),27.723,88.064,19.7,6.9 Mw,L,NaN,NaN,NaN,NaN,NaN,6.9,Mw
1338,1338,2011-09-23 10:41,Turkey (see 2011 Van earthquake),38.6,43.5,7.2,7.1 Mw,NaN,NaN,NaN,NaN,NaN,NaN,7.1,Mw


In [4]:
wiki['country']= wiki['country'].str.replace(
    '( \(.*\)?)','').str.replace(
    '(\(see 1997 Cariaco earthquake\))','Venezuela').str.replace(
    '(Iran, 2005 Qeshm earthquake)','Iran').str.replace(
    'US Territory of Alaska','United States').str.replace(
    'Mandatory Palestine and Transjordan','Palestine').str.replace(
    'Burma','Myanmar').str.replace(
    'Tajik Soviet Socialist Republic','Tajikistan').str.replace(
    'Kingdom of Yugoslavia','Yugoslavia'
)

In [5]:
wiki[['magnitude','emdat_total_deaths']]= wiki[['magnitude','emdat_total_deaths']].apply(lambda x: x.str.replace('( \[\d{1,2}\])',''), axis=1)

In [6]:
wiki['year'] = wiki['origin'].str[:4]
wiki.head()

,Unnamed: 0,origin,country,lat,long,depth_km,magnitude,secondary_effects,pde_shaking_deaths,pde_total_deaths,utsu_total_deaths,emdat_total_deaths,deaths,magnitude.1,measure,year
0,0,1900-05-11 17:23,Japan,38.700,141.100,5,7.0 MJMA,NaN,NaN,NaN,NaN,NaN,NaN,7.0,MJMA,1900
1,1,1900-07-12 06:25,Turkey,40.300,43.100,NaN,5.9 Muk,NaN,NaN,NaN,140.0,NaN,140.0,5.9,Muk,1900
2,2,1900-10-29 09:11,Venezuela,11.000,-66.000,0,7.7 Mw,NaN,NaN,NaN,NaN,NaN,NaN,7.7,Mw,1900
3,3,1901-02-15 00:00,China,26.000,100.100,0,6.5 Ms,NaN,NaN,NaN,NaN,NaN,NaN,6.5,Ms,1901
4,4,1901-03-31 07:11,Bulgaria,43.400,28.700,NaN,6.4 Muk,NaN,NaN,NaN,4.0,NaN,4.0,6.4,Muk,1901


In [7]:
quake_deaths = wiki[wiki['deaths'].notna()].groupby(
    'country')[['country', 'deaths']].agg(['count', 'sum', np.mean, 'median', 'max'])
quake_deaths

deaths                                         
                   count      sum          mean   median      max
country                                                          
Afghanistan           28   9338.0    333.500000      8.0   4000.0
Albania                7    244.0     34.857143     25.0    120.0
Algeria               19   9330.0    491.052632      9.0   5000.0
Argentina             11   8584.0    780.363636      8.0   8000.0
Armenia                2  25390.0  12695.000000  12695.0  25000.0
...                  ...      ...           ...      ...      ...
Venezuela             10    442.0     44.200000      4.0    240.0
Vietnam                1      1.0      1.000000      1.0      1.0
Wallis and Futuna      1      5.0      5.000000      5.0      5.0
Yemen                  3   4011.0   1337.000000   1200.0   2800.0
Yugoslavia             2     47.0     23.500000     23.5     35.0

[100 rows x 5 columns]

In [8]:
quake_deaths.columns = ['count', 'sum', 'mean', 'median', 'max']

In [9]:
quake_deaths.sort_values('count', ascending=False).head(10)

,count,sum,mean,median,max
country,,,,,
China,145,1138388.0,7850.951724,12.0,655237.0
Iran,90,152677.0,1696.411111,21.0,45000.0
Turkey,89,90836.0,1020.629213,23.0,32968.0
Indonesia,86,342112.0,3978.046512,15.5,297200.0
Japan,76,166997.0,2197.328947,8.5,142807.0
Peru,52,73941.0,1421.942308,10.5,70000.0
Taiwan,48,7954.0,165.708333,5.0,3276.0
Mexico,42,12706.0,302.523810,19.0,9500.0
Chile,39,39461.0,1011.820513,3.0,28000.0


In [10]:
quake_deaths.sort_values('sum', ascending=False).head(10)

,count,sum,mean,median,max
country,,,,,
China,145,1138388.0,7850.951724,12.0,655237.0
Indonesia,86,342112.0,3978.046512,15.5,297200.0
Japan,76,166997.0,2197.328947,8.5,142807.0
Pakistan,23,153910.0,6691.739130,19.0,87351.0
Iran,90,152677.0,1696.411111,21.0,45000.0
Italy,33,121423.0,3679.484848,5.0,82000.0
Turkey,89,90836.0,1020.629213,23.0,32968.0
Peru,52,73941.0,1421.942308,10.5,70000.0
India,23,52354.0,2276.260870,15.0,20023.0


In [11]:
quake_deaths.sort_values('mean', ascending=False).head(10)

,count,sum,mean,median,max
country,,,,,
Armenia,2,25390.0,12695.000000,12695.0,25000.0
China,145,1138388.0,7850.951724,12.0,655237.0
Pakistan,23,153910.0,6691.739130,19.0,87351.0
Turkmenistan,4,23468.0,5867.000000,1828.5,19800.0
Indonesia,86,342112.0,3978.046512,15.5,297200.0
Italy,33,121423.0,3679.484848,5.0,82000.0
Morocco,4,13735.0,3433.750000,316.5,13100.0
Tajikistan,8,27324.0,3415.500000,20.5,15000.0
Nepal,4,12418.0,3104.500000,819.0,10700.0


In [12]:
quake_deaths.sort_values('median', ascending=False).head(10)

,count,sum,mean,median,max
country,,,,,
Armenia,2,25390.0,12695.00,12695.0,25000.0
Turkmenistan,4,23468.0,5867.00,1828.5,19800.0
Yemen,3,4011.0,1337.00,1200.0,2800.0
Macedonia,1,1100.0,1100.00,1100.0,1100.0
Nepal,4,12418.0,3104.50,819.0,10700.0
Guinea,1,643.0,643.00,643.0,643.0
Jamaica,2,1001.0,500.50,500.5,1000.0
Morocco,4,13735.0,3433.75,316.5,13100.0
Palestine,2,381.0,190.50,190.5,361.0


In [13]:
quake_deaths.sort_values('max', ascending=False).head(10)

,count,sum,mean,median,max
country,,,,,
China,145,1138388.0,7850.951724,12.0,655237.0
Indonesia,86,342112.0,3978.046512,15.5,297200.0
Japan,76,166997.0,2197.328947,8.5,142807.0
Pakistan,23,153910.0,6691.739130,19.0,87351.0
Italy,33,121423.0,3679.484848,5.0,82000.0
Peru,52,73941.0,1421.942308,10.5,70000.0
Iran,90,152677.0,1696.411111,21.0,45000.0
Turkey,89,90836.0,1020.629213,23.0,32968.0
Chile,39,39461.0,1011.820513,3.0,28000.0


## Import and Clean Population Density

In [14]:
density = pd.read_csv('../data/population-density-3.csv')
density

,Entity,Code,Year,Population Density ((HYDE) and UN Population Division (2008 Revision))
0,Afghanistan,AFG,-10000,0.023107
1,Afghanistan,AFG,-5000,0.117594
2,Afghanistan,AFG,0,3.135838
3,Afghanistan,AFG,500,3.919798
4,Afghanistan,AFG,1000,3.527818
...,...,...,...,...
9739,Zimbabwe,ZWE,2080,59.050135
9740,Zimbabwe,ZWE,2085,59.243638
9741,Zimbabwe,ZWE,2090,59.393715
9742,Zimbabwe,ZWE,2095,59.500026


In [15]:
density = density[(density['Year']>=1900) & (density['Year']<=2020)]
density.columns = ['country', 'code', 'year', 'density']
density = density.drop('code', axis = 1)
density.tail(10)

,country,year,density
9718,Zimbabwe,1975,15.854651
9719,Zimbabwe,1980,18.719331
9720,Zimbabwe,1985,22.735189
9721,Zimbabwe,1990,26.890409
9722,Zimbabwe,1995,30.109275
9723,Zimbabwe,2000,32.016621
9724,Zimbabwe,2005,32.067316
9725,Zimbabwe,2010,32.501622
9726,Zimbabwe,2015,36.061062
9727,Zimbabwe,2020,40.025440


In [16]:
density.year = density.year.astype(int)

In [17]:
density['year'].value_counts().sort_index()

1900    232
1950    232
1955    232
1960    232
1965    232
1970    232
1975    232
1980    232
1985    232
1990    232
1995    232
2000    232
2005    232
2010    232
2015    232
2020    232
Name: year, dtype: int64

## Import and Clean GDP

In [18]:
gdp_df = pd.read_csv("../data/gdp_per_capita.csv")[:-2]
gdp_df = gdp_df.drop('Value Footnotes', axis = 1)
gdp_df.columns = ['country', 'year', 'gdp_per_capita']
gdp_df['year'] = gdp_df.year.astype(int)
gdp_df

,country,year,gdp_per_capita
0,Afghanistan,2018,1734.723214
1,Afghanistan,2017,1758.465636
2,Afghanistan,2016,1757.023490
3,Afghanistan,2015,1766.593077
4,Afghanistan,2014,1795.735834
...,...,...,...
6724,Zimbabwe,1994,2768.309953
6725,Zimbabwe,1993,2572.870395
6726,Zimbabwe,1992,2591.007534
6727,Zimbabwe,1991,2906.272849


## Merge DataFrames

In [19]:
gdp_density = gdp_df.merge(density, on=['country', 'year'], how='left')
gdp_density = gdp_density.sort_values(['country', 'year'], ascending=[False, True]).reset_index()
gdp_density

,index,country,year,gdp_per_capita,density
0,6728,Zimbabwe,1990,2819.549467,26.890409
1,6727,Zimbabwe,1991,2906.272849,NaN
2,6726,Zimbabwe,1992,2591.007534,NaN
3,6725,Zimbabwe,1993,2572.870395,NaN
4,6724,Zimbabwe,1994,2768.309953,NaN
...,...,...,...,...,...
6724,4,Afghanistan,2014,1795.735834,NaN
6725,3,Afghanistan,2015,1766.593077,53.694768
6726,2,Afghanistan,2016,1757.023490,NaN
6727,1,Afghanistan,2017,1758.465636,NaN


In [20]:
density = None
country = None
for row in gdp_density.itertuples(): 
    if pd.notna(row.density):
        density = row.density
        country = row.country
    else: 
        if gdp_density.at[row.Index, 'country'] == country:
            gdp_density.at[row.Index, 'density'] = density
            continue
        for i in range(1,5):
            if pd.notna(gdp_density.at[row.Index+i, 'density']):
                gdp_density.at[row.Index, 'density'] = gdp_density.at[row.Index+i, 'density']
                break

## MAPPING

In [21]:
#Need to drop columns with NaNs in lat & long, and index 413, which has ?? in lat/long.
wiki = wiki[wiki['lat'].notna()]
wiki = wiki[wiki['long'].notna()]
wiki = wiki.drop(wiki.index[413])

In [22]:
wiki

,Unnamed: 0,origin,country,lat,long,depth_km,magnitude,secondary_effects,pde_shaking_deaths,pde_total_deaths,utsu_total_deaths,emdat_total_deaths,deaths,magnitude.1,measure,year
0,0,1900-05-11 17:23,Japan,38.700,141.100,5,7.0 MJMA,NaN,NaN,NaN,NaN,NaN,NaN,7.0,MJMA,1900
1,1,1900-07-12 06:25,Turkey,40.300,43.100,NaN,5.9 Muk,NaN,NaN,NaN,140.0,NaN,140.0,5.9,Muk,1900
2,2,1900-10-29 09:11,Venezuela,11.000,-66.000,0,7.7 Mw,NaN,NaN,NaN,NaN,NaN,NaN,7.7,Mw,1900
3,3,1901-02-15 00:00,China,26.000,100.100,0,6.5 Ms,NaN,NaN,NaN,NaN,NaN,NaN,6.5,Ms,1901
4,4,1901-03-31 07:11,Bulgaria,43.400,28.700,NaN,6.4 Muk,NaN,NaN,NaN,4.0,NaN,4.0,6.4,Muk,1901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1328,1328,2010-02-27 06:34,Chile,-35.846,-72.719,35,8.8 Mw,T,NaN,NaN,NaN,NaN,NaN,8.8,Mw,2010
1334,1334,2011-03-11 14:46,Japan,38.322,142.369,24.4,9.0 Mw,T,NaN,NaN,NaN,NaN,NaN,9.0,Mw,2011
1336,1336,2011-04-07 14:32,Japan,38.2,140.0,66.0,7.1 Mw,NaN,NaN,NaN,NaN,NaN,NaN,7.1,Mw,2011
1337,1337,2011-09-18 12:40,India,27.723,88.064,19.7,6.9 Mw,L,NaN,NaN,NaN,NaN,NaN,6.9,Mw,2011


In [23]:
wiki['geometry'] = wiki.apply(lambda x: Point((float(x.long),
                                              float(x.lat))),
                             axis=1)

In [24]:
center = [0, 0]
quake_map = folium.Map(location = center, zoom_start=2)

In [25]:
for row_index, row_values in wiki.iterrows():
    loc = [row_values['lat'], row_values['long']]
    pop = str(row_values['origin'])
    marker = folium.Marker(
        location = loc,
        popup = pop)
    
    marker.add_to(quake_map)

quake_map

In [26]:
cluster_quake_map = folium.Map(location = center, zoom_start=2)
marker_cluster = MarkerCluster().add_to(cluster_quake_map)

for row_index, row_values in wiki.iterrows():
    loc = [row_values['lat'], row_values['long']]
    pop = str(row_values['origin'])
    marker = folium.Marker(
        location = loc, 
        popup = pop)
    
    marker.add_to(marker_cluster)
    
cluster_quake_map

In [27]:
cities = pd.read_csv('../data/worldcities.csv')

In [28]:
cities[cities['population']<10000]

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
818,Valletta,Valletta,35.8978,14.5125,Malta,MT,MLT,Valletta,primary,6444.0,1470574399
819,Palikir,Palikir,6.9178,158.1850,"Micronesia, Federated States Of",FM,FSM,Pohnpei,primary,6227.0,1583008885
820,Funafuti,Funafuti,-8.5167,179.2167,Tuvalu,TV,TUV,Funafuti,primary,6025.0,1798251391
821,Lobamba,Lobamba,-26.4167,31.1667,Swaziland,SZ,SWZ,Hhohho,primary,5800.0,1748963140
822,Saint George’s,Saint George's,12.0444,-61.7417,Grenada,GD,GRD,Saint George,primary,4315.0,1308891766
...,...,...,...,...,...,...,...,...,...,...,...
26557,Nord,Nord,81.7166,-17.8000,Greenland,GL,GRL,Sermersooq,NaN,10.0,1304217709
26558,Timmiarmiut,Timmiarmiut,62.5333,-42.2167,Greenland,GL,GRL,Kujalleq,NaN,10.0,1304206491
26559,Cheremoshna,Cheremoshna,51.3894,30.0989,Ukraine,UA,UKR,Kyyivs’ka Oblast’,NaN,0.0,1804043438
26560,Ambarchik,Ambarchik,69.6510,162.3336,Russia,RU,RUS,Sakha (Yakutiya),NaN,0.0,1643739159
